In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from random import choice
from sklearn.utils import shuffle

# The main class

In [2]:
def proxy_generator():
    response = requests.get("https://sslproxies.org/")
    soup = BeautifulSoup(response.content, 'html5lib')
    proxy = {'https': str(choice(list(map(lambda x:x[0]+':'+x[1], list(zip(map(lambda x:x.text, soup.findAll('td')[::8]), map(lambda x:x.text, soup.findAll('td')[1::8])))))))}
    
    print("--->")
    print(proxy)
    return proxy

def data_scraper(request_method, url, **kwargs):
    while True:
        try:
            proxy = proxy_generator()
            print("Proxy currently being used: {}".format(proxy))
            response = requests.request(request_method, url, proxies=proxy, timeout=7, **kwargs)
            break
            # if the request is successful, no exception is raised
        except:
            print("Connection error, looking for another proxy")
            pass
    return response

In [15]:
def processLabel(label):
    label = label.replace(":","")
    label = label.replace('\\n','')
    return label

def tag_class_redunduncy(value, depth, data):
    output = ""
    if(depth != len(value)):
       
        data = data[0].find_all(class_=value[depth])
      
        depth += 1
        
        if(len(data)!=0):
            return tag_class_redunduncy(value, depth, [data[0]])
    else:
        if(len(data)!=0):
            output = data[0].get_text()
        else:
            output = "no_value_p"
    
    return output
    
    
def tag_multi_class_redunduncy(value, depth, data, data_value, output_label, output_value):
    for _data in data:
        if (depth <= len(value)-1):
            datas = _data.find_all(class_ = value[depth])
            depth +=1
            tag_multi_class_redunduncy(value, depth, datas, data_value, output_label, output_value)
        else:
            a = _data.get_text()
            value_parent = _data.parent
            value = value_parent.find_all(class_=data_value[0])
            output_label.append(a)
            if(len(value)>0):
                output_value.append(value[0].get_text())
            else:
                output_value.append("no_value_p")

def scrap(document, where_to_find, general_data, df_output):
    output = []
    # completing all label
    for i in range(0, len(where_to_find)):
        if (where_to_find[i]["label"]==""):
            if(len(where_to_find[i]["ressources"])==1):
                
                data = document.find_all(class_ = where_to_find[i]["ressources"])
                
                for datas in data:
                    if (len(where_to_find[i]["values"])==1):
                        value_parent = datas.parent
                        value_tag = value_parent.find_all(class_=where_to_find[i]["values"])
                        
                        if(len(value_tag)!=0):
                            df_output[datas.get_text()] = [value_tag[0].get_text()]
                        
                    else:
                        for values in where_to_find[i]["values"]:
                            value_parent = datas.parent
                            value_tag = value_parent.find_all(class_=values)
                            if(len(value_tag)!=0):
                                df_output[datas.get_text()] = [value_tag[0].get_text()]
                    
            else:
                my_add_labels = []
                my_add_values = []
                tag_multi_class_redunduncy(where_to_find[i]["ressources"], 0, [document],
                                           where_to_find[i]["values"], my_add_labels, my_add_values)  
                
                for i in range(0, len(my_add_labels)):
                    
                    df_output[my_add_labels[i]] = [my_add_values[i]]
                
        else:
            if(len(where_to_find[i]["ressources"])>1):
               
                df_output[where_to_find[i]["label"]] = [tag_class_redunduncy(
                        where_to_find[i]["ressources"],
                        0,
                        [document]
                    )]
            else:
                value_tag = document.find_all(class_=where_to_find[i]["ressources"][0])
                
                if(len(value_tag)!=0):
                    df_output[where_to_find[i]["label"]] = [value_tag[0].get_text()]
                else:
                    df_output[where_to_find[i]["label"]] = ["no_value_p"]
                
                
    df_output["province"]=[general_data[0]]
    df_output["city"]=[general_data[1]]
    df_output["suburbaine"]=[general_data[2]]
    
    return df_output

            
                

def get_data_by_url(url, general_data, output):

    my_data = any
    
    #using default request
    #result = requests.get(url)
    
    # using bee
    # client_bee = ScrapingBeeClient(api_key=token)
    result = data_scraper("get",url)
    print(result.status_code)
    if(result.status_code==200):
        src = result.content
        document = BeautifulSoup(src, 'lxml')
    
        where_to_find = [
        {
            "label":"price",
            "ressources":["pull-left","p24_price"]
        },
        {
            "label":"",
            "ressources":["p24_propertyOverviewKey"],
            "values": ["p24_info","p24_addressPropOverview"]
        },
        {
            "label":"",
            "ressources":["p24_listingFeatures","p24_feature"],
            "values":["p24_featureAmount"]
        },
        {
            "label":"address",
            "ressources":["p24_mB","p24_address"]
        },
        {
            "label":"description",
            "ressources":["p24_listingAbout"]
        }
        ]
    
        my_data = scrap(document, where_to_find, general_data, output)
    else:
        my_data = pd.DataFrame()
    return my_data

# Load all urls

In [16]:
list_urls = pd.read_csv("list_url_final_shuffled.csv")

In [17]:
list_urls['url'][0][:len(list_urls['url'][0]) -2]

'https://property24.com/for-sale/buccleuch/sandton/gauteng/5255/110201915'

# Begin the scrap

In [23]:
# begin the scrap

real_data = []

# write here the number of urls you done
begin_i  = 300
last_i = 1000

# create the file 
filename = "Datas/data-%d-%d.pkl" %(begin_i, last_i)
emptyDf = pd.DataFrame()
emptyDf.to_pickle(filename)

for i in range(begin_i, last_i):
    print(str(i), "----> ")
    every_data = pd.DataFrame({},index=[i])
    
    real_data.append(get_data_by_url(list_urls["url"][i][:len(list_urls['url'][i]) -2], [
            list_urls["province"][i],
            list_urls["city"][i],
            list_urls["suburbaine"][i]
        ] , every_data))
    
    my_dataframe = pd.concat(real_data)
    my_dataframe.to_pickle(filename)
    

300 ----> 
--->
{'https': '47.75.254.142:8888'}
Proxy currently being used: {'https': '47.75.254.142:8888'}
Connection error, looking for another proxy
--->
{'https': '190.82.91.203:999'}
Proxy currently being used: {'https': '190.82.91.203:999'}
Connection error, looking for another proxy
--->
{'https': '176.56.107.153:56667'}
Proxy currently being used: {'https': '176.56.107.153:56667'}
Connection error, looking for another proxy
--->
{'https': '46.229.67.198:47437'}
Proxy currently being used: {'https': '46.229.67.198:47437'}
200
301 ----> 
--->
{'https': '61.37.223.152:8080'}
Proxy currently being used: {'https': '61.37.223.152:8080'}
Connection error, looking for another proxy
--->
{'https': ' FR:76'}
Proxy currently being used: {'https': ' FR:76'}
Connection error, looking for another proxy
--->
{'https': '180.245.156.243:8080'}
Proxy currently being used: {'https': '180.245.156.243:8080'}
200


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



302 ----> 
--->
{'https': '167.71.199.228:8080'}
Proxy currently being used: {'https': '167.71.199.228:8080'}
Connection error, looking for another proxy
--->
{'https': '116.73.14.16:8080'}
Proxy currently being used: {'https': '116.73.14.16:8080'}
Connection error, looking for another proxy
--->
{'https': '213.230.68.208:3128'}
Proxy currently being used: {'https': '213.230.68.208:3128'}
Connection error, looking for another proxy
--->
{'https': '212.109.196.159:8118'}
Proxy currently being used: {'https': '212.109.196.159:8118'}
Connection error, looking for another proxy
--->
{'https': '114.199.80.100:8182'}
Proxy currently being used: {'https': '114.199.80.100:8182'}
Connection error, looking for another proxy
--->
{'https': '187.111.160.6:8080'}
Proxy currently being used: {'https': '187.111.160.6:8080'}
Connection error, looking for another proxy
--->
{'https': '188.168.75.254:56899'}
Proxy currently being used: {'https': '188.168.75.254:56899'}
Connection error, looking for anot

--->
{'https': '149.56.86.231:80'}
Proxy currently being used: {'https': '149.56.86.231:80'}
Connection error, looking for another proxy
--->
{'https': '190.82.91.203:999'}
Proxy currently being used: {'https': '190.82.91.203:999'}
Connection error, looking for another proxy
--->
{'https': '187.243.255.174:8080'}
Proxy currently being used: {'https': '187.243.255.174:8080'}
Connection error, looking for another proxy
--->
{'https': ' JP:19'}
Proxy currently being used: {'https': ' JP:19'}
Connection error, looking for another proxy
--->
{'https': '149.56.86.231:80'}
Proxy currently being used: {'https': '149.56.86.231:80'}
Connection error, looking for another proxy
--->
{'https': '149.56.86.231:80'}
Proxy currently being used: {'https': '149.56.86.231:80'}
Connection error, looking for another proxy
--->
{'https': ' BR:9'}
Proxy currently being used: {'https': ' BR:9'}
Connection error, looking for another proxy
--->
{'https': '103.217.173.210:53905'}
Proxy currently being used: {'htt

Connection error, looking for another proxy
--->
{'https': '45.42.177.72:3128'}
Proxy currently being used: {'https': '45.42.177.72:3128'}
Connection error, looking for another proxy
--->
{'https': '89.250.149.114:60981'}
Proxy currently being used: {'https': '89.250.149.114:60981'}
Connection error, looking for another proxy
--->
{'https': ' MX:13'}
Proxy currently being used: {'https': ' MX:13'}
Connection error, looking for another proxy
--->
{'https': 'S.A.:12'}
Proxy currently being used: {'https': 'S.A.:12'}
Connection error, looking for another proxy
--->
{'https': '51.15.8.75:3738'}
Proxy currently being used: {'https': '51.15.8.75:3738'}
Connection error, looking for another proxy
--->
{'https': '103.217.173.210:53905'}
Proxy currently being used: {'https': '103.217.173.210:53905'}
Connection error, looking for another proxy
--->
{'https': '187.111.160.6:8080'}
Proxy currently being used: {'https': '187.111.160.6:8080'}
Connection error, looking for another proxy
--->
{'https'

...

--->
{'https': '157.90.141.221:80'}
Proxy currently being used: {'https': '157.90.141.221:80'}
Connection error, looking for another proxy
--->
{'https': '12.218.209.130:53281'}
Proxy currently being used: {'https': '12.218.209.130:53281'}
Connection error, looking for another proxy
--->
{'https': '157.90.141.221:80'}
Proxy currently being used: {'https': '157.90.141.221:80'}
Connection error, looking for another proxy
--->
{'https': '112.78.161.58:43318'}
Proxy currently being used: {'https': '112.78.161.58:43318'}
Connection error, looking for another proxy
--->
{'https': ' CH:23'}
Proxy currently being used: {'https': ' CH:23'}
Connection error, looking for another proxy
--->
{'https': ' CA:169'}
Proxy currently being used: {'https': ' CA:169'}
Connection error, looking for another proxy
--->
{'https': '103.103.212.222:53281'}
Proxy currently being used: {'https': '103.103.212.222:53281'}
Connection error, looking for another proxy
--->
{'https': '103.251.214.167:6666'}
Proxy curren

--->
{'https': '210.2.135.254:3128'}
Proxy currently being used: {'https': '210.2.135.254:3128'}
Connection error, looking for another proxy
--->
{'https': '45.230.169.10:999'}
Proxy currently being used: {'https': '45.230.169.10:999'}
Connection error, looking for another proxy
--->
{'https': '106.104.148.208:80'}
Proxy currently being used: {'https': '106.104.148.208:80'}
Connection error, looking for another proxy
--->
{'https': '202.62.67.209:53281'}
Proxy currently being used: {'https': '202.62.67.209:53281'}
200
974 ----> 
--->
{'https': '85.15.69.150:36652'}
Proxy currently being used: {'https': '85.15.69.150:36652'}
Connection error, looking for another proxy
--->
{'https': '124.106.224.5:8080'}
Proxy currently being used: {'https': '124.106.224.5:8080'}
Connection error, looking for another proxy
--->
{'https': ' MX:13'}
Proxy currently being used: {'https': ' MX:13'}
Connection error, looking for another proxy
--->
{'https': '34.121.55.127:8080'}
Proxy currently being used: {

--->
{'https': '195.154.106.59:5566'}
Proxy currently being used: {'https': '195.154.106.59:5566'}
Connection error, looking for another proxy
--->
{'https': '167.71.199.228:8080'}
Proxy currently being used: {'https': '167.71.199.228:8080'}
Connection error, looking for another proxy
--->
{'https': '35.239.150.193:8080'}
Proxy currently being used: {'https': '35.239.150.193:8080'}
Connection error, looking for another proxy
--->
{'https': '185.156.35.156:57367'}
Proxy currently being used: {'https': '185.156.35.156:57367'}
Connection error, looking for another proxy
--->
{'https': ' FR:76'}
Proxy currently being used: {'https': ' FR:76'}
Connection error, looking for another proxy
--->
{'https': '50.233.42.98:51696'}
Proxy currently being used: {'https': '50.233.42.98:51696'}
Connection error, looking for another proxy
--->
{'https': '172.245.124.125:3128'}
Proxy currently being used: {'https': '172.245.124.125:3128'}
Connection error, looking for another proxy
--->
{'https': '8.210.1

--->
{'https': '86.34.197.6:23500'}
Proxy currently being used: {'https': '86.34.197.6:23500'}
Connection error, looking for another proxy
--->
{'https': '140.227.64.163:3128'}
Proxy currently being used: {'https': '140.227.64.163:3128'}
Connection error, looking for another proxy
--->
{'https': '103.41.212.227:44759'}
Proxy currently being used: {'https': '103.41.212.227:44759'}
Connection error, looking for another proxy
--->
{'https': '206.0.15.69:8080'}
Proxy currently being used: {'https': '206.0.15.69:8080'}
Connection error, looking for another proxy
--->
{'https': '104.128.228.69:8118'}
Proxy currently being used: {'https': '104.128.228.69:8118'}
Connection error, looking for another proxy
--->
{'https': '36.89.187.193:43032'}
Proxy currently being used: {'https': '36.89.187.193:43032'}
Connection error, looking for another proxy
--->
{'https': '113.160.119.161:55443'}
Proxy currently being used: {'https': '113.160.119.161:55443'}
Connection error, looking for another proxy
---

503
994 ----> 
--->
{'https': ' CH:23'}
Proxy currently being used: {'https': ' CH:23'}
Connection error, looking for another proxy
--->
{'https': '46.229.187.169:53281'}
Proxy currently being used: {'https': '46.229.187.169:53281'}
Connection error, looking for another proxy
--->
{'https': '106.104.148.208:80'}
Proxy currently being used: {'https': '106.104.148.208:80'}
Connection error, looking for another proxy
--->
{'https': '1.10.141.220:54620'}
Proxy currently being used: {'https': '1.10.141.220:54620'}
Connection error, looking for another proxy
--->
{'https': '62.33.210.34:58918'}
Proxy currently being used: {'https': '62.33.210.34:58918'}
Connection error, looking for another proxy
--->
{'https': ' MX:13'}
Proxy currently being used: {'https': ' MX:13'}
Connection error, looking for another proxy
--->
{'https': '195.154.106.59:5566'}
Proxy currently being used: {'https': '195.154.106.59:5566'}
503
995 ----> 
--->
{'https': '150.129.148.88:35101'}
Proxy currently being used: {'

In [16]:
all_data = pd.concat(real_data)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [17]:
all_data

,Bathrooms: 1,Fibre Internet,Garden,Age,Backup Water,Bathroom,Bathrooms,Bedroom,Bedrooms,Bedrooms:,...,Street Address,Temperature Control,Type of Property,Wheelchair Accessible,address,city,description,price,province,suburbaine
0,2,NaN,no_value_p,NaN,NaN,NaN,1,NaN,2,2,...,"20 Argyle Ave, Buccleuch",NaN,Apartment / Flat,NaN,"20 Argyle Ave, Buccleuch, Sandton",sandton,\nSpacious 2 Bedroom Apartment \n\nA perfect s...,R 750 000,gauteng,buccleuch
1,1,no_value_p,NaN,1,No,"Tiled Floors, Shower, Toilet and Basin",NaN,"Balcony, Built-in Cupboards",NaN,1,...,NaN,Air Conditioning Unit,Apartment / Flat,Yes,,cape-town,\nUnfurnished corner 1 bedroom with 2 balconie...,R 4 355 000,western-cape,cape-town-city-centre


In [9]:
filename = "Datas/data-0-1000.pkl"

In [12]:
data = pd.read_pickle(filename)

print(data.shape)

(158, 161)


In [21]:
len(data.values.tolist())

158